# <center> CREAR ARCHIVOS</center>

In [1]:
def crear_fecha():
    
    import time
    from datetime import datetime

    # detiene el tiempo para que no todos los archivos generados queden con la misma fecha de timestamp
    time.sleep(0.1)
        
    # Getting the current date and time
    dt = datetime.now()

    # getting the timestamp
    ts = datetime.timestamp(dt)

    return ts

def generar_archivos(todos_los_archivos=True):

    lista_de_archivos = ['info.txt','Dockerfile','Manifest.yaml','imagen.png','main.py','connection.java','calculos.c','addon.php','utils.gif','query.sql','dataset.avro','link.vs']
    
    lista_de_archivos = lista_de_archivos if todos_los_archivos else [lista_de_archivos[0]]
    
    print('lista_de_archivos = {lista_de_archivos}'.format(lista_de_archivos=lista_de_archivos))
    
    for archivo in lista_de_archivos:
        comando = 'echo {crear_archivo} >> {archivo}'.format(crear_archivo=crear_fecha(), archivo=archivo)
        ejecutar_comando_shell(comando)

    return lista_de_archivos


def crear_commits(total_commits=5):
    
    import random
    
    for commit in range(total_commits):
        
        numero_aleatorio = random.randint(1, 10_000_000)
        generar_archivos()

        comando = 'git status'
        ejecutar_comando_git(comando)

        comando = 'git add .'
        ejecutar_comando_git(comando)

        comando = 'git status'
        ejecutar_comando_git(comando)

        comando = 'git commit -m "PRUEBA numero {numero_aleatorio} de adicion de archivos y creacion de COMMIT {fecha}"'.format( numero_aleatorio=numero_aleatorio, fecha=crear_fecha())
        ejecutar_comando_git(comando)

    comando = 'git log --oneline'
    ejecutar_comando_git(comando)

    comando = 'git status'
    ejecutar_comando_git(comando)

# <center> ELIMINAR ARCHIVOS</center>

In [2]:
def listar_archivos_eliminables_en_directorio():
    import os
    
    listado_de_archivos = os.listdir(directory)
    
    archivos_eliminables = [archivo for archivo in listado_de_archivos if '.git' not in archivo]
    
    return archivos_eliminables
    
def eliminar_directorio(nombre_repositorio):
    
    import shutil
    import os
    import stat
    from os import path
    
    for root, dirs, files in os.walk("./{nombre_repositorio}".format(nombre_repositorio=nombre_repositorio)):  
        for dir in dirs:
            try:
                os.chmod(path.join(root, dir), stat.S_IRWXU)
            except:
                print('el archivo {archivo} no existe'.format(archivo=archivo))                
        for file in files:
            try:
                os.chmod(path.join(root, file), stat.S_IRWXU)
            except:
                print('el archivo {archivo} no existe'.format(archivo=archivo))
    try: 
        shutil.rmtree("./{nombre_repositorio}".format(nombre_repositorio=nombre_repositorio))
    except:
        print('el repositorio {nombre_repositorio} no existe'.format(nombre_repositorio=nombre_repositorio))

def eliminar_archivos_directorio():
    import os
    
    lista_de_archivos_eliminables = listar_archivos_eliminables_en_directorio()
    
    for archivo in lista_de_archivos_eliminables:
            os.remove(archivo)
        
    return 'Eliminacion Completada con Exito!!! de {lista_de_archivos_eliminables}'.format(lista_de_archivos_eliminables=lista_de_archivos_eliminables)

# <center> CREAR ARCHIVOS, COMMITS Y PUBLICAR RAMAS AL REPOSITORIO </center>

In [3]:
def validar_si_existe_rama(nombre_rama, local=False):
    
    comando ='git branch -a'
    resultado = ejecutar_comando_shell(comando, response=True)
    
    rama_base = '{nombre_rama}'.format(nombre_rama=nombre_rama) if local else 'origin/{nombre_rama}'.format(nombre_rama=nombre_rama)

    if rama_base in resultado:
        return True
    return False

def elimina_ramas_si_existen_y_crea_la_rama_local(nombre_rama):
    
    # valida si las ramas existen en REMOTO y LOCAL
    rama_remota = validar_si_existe_rama(nombre_rama=nombre_rama)
    rama_local = validar_si_existe_rama(nombre_rama=nombre_rama, local=True)
    
    # verifica si existe la rama la elimina
    if rama_remota:
        # elimina rama REMOTA existente
        comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)

     # verifica si existe la rama la elimina
    if rama_local:                                                                                                            
        # elimina la rama
        comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)
        
    # crea la rama
    comando = 'git checkout -b {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_shell(comando)
    
    # verifica las ramas actuales
    ejecutar_comando_shell()
    

def crear_commit_por_cada_archivo(todos_los_archivos):
    
    import datetime

    fecha_ahora = datetime.datetime.now()
    
    # crea el listado de archivos
    lista_de_archivos = generar_archivos(todos_los_archivos=todos_los_archivos)
    
    # basado en el listado de archivos, adiciona uno a uno y crea un commit por cada uno
    for archivo in lista_de_archivos:

        comando = 'git add {archivo}'.format(archivo=archivo)
        ejecutar_comando_git(comando)
        
        comando = 'git commit -m "ADICIONADO ARCHIVO {archivo} {fecha_ahora}"'.format(archivo=archivo, fecha_ahora=fecha_ahora)
        ejecutar_comando_git(comando)

    # verifica si todos los archivos estan en los commits creados
    comando = 'git status'
    ejecutar_comando_git(comando)
    
def generar_archivos_y_hacer_commit(nombre_rama, todos_los_archivos = True):
    
    # elimina cambios a la rama remota si los tiene
    comando = 'git restore .'
    ejecutar_comando_git(comando)
    
    # cambiar a la rama remota
    comando = 'git checkout {nombre_rama_remota}'.format(nombre_rama_remota=nombre_rama_remota)
    ejecutar_comando_git(comando)
    
    # valida si las ramas existen en REMOTO y LOCAL y las elimina
    elimina_ramas_si_existen_y_crea_la_rama_local(nombre_rama)
    
    # adiciona archivos de 1 en 1 y genera 1 commit por cada archivo
    crear_commit_por_cada_archivo(todos_los_archivos=todos_los_archivos)

    # mostrar el log de las ramas en forma resumida y con diagrama de uniones como grafo 
    comando = 'git log --graph --oneline'
    ejecutar_comando_git(comando)

    # crea la rama en remoto
    comando = 'git push --set-upstream origin {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)
    
    ejecutar_comando_shell()

# <center> OBTENER LISTA CON LOS NOMBRES DE TODAS LAS RAMAS LOCALES DE UN REPOSITORIO GIT</center>

In [4]:
def obtener_lista_nombres_ramas_git():

    stdout = subprocess.check_output('git branch'.split())
    out = stdout.decode()
    branches = [b.strip('* ') for b in out.splitlines()]
    return branches

# <center> EJECUTAR COMANDOS SHELL DESDE FUNCION PYTHON</center>

In [5]:
import subprocess

def ejecutar_comando_shell(comando='git branch', response=False):

    resultado_comando_ejecutado = subprocess.check_output(comando, shell=True, text=True)
    print('############################################# COMANDO A EJECUTAR #############################################')
    print(comando)
    print('####################################### RESULTADO DE COMANDO EJECUTADO #######################################')
    #para evitar el salto de linea usa el parametro end=''
    print(resultado_comando_ejecutado, end='')
    print('##############################################################################################################')
    print()
    if response:
        return resultado_comando_ejecutado

# <center> FUNCION LAMBDA PARA EJECUTAR COMANDOS GIT</center>

In [6]:
ejecutar_comando_git = lambda comando : ejecutar_comando_shell(comando)

# <center> CICLO DE VIDA DE UN REPOSITORIO GIT</center>

<center>
    <img width="100%" align="center" src="git_life_cycle.png">
</center>

# <center> FLUJO DE ARCHIVOS DE UN REPOSITORIO GIT</center>

<center>
    <img width="100%" align="center" src="gitflow.png">
</center>

# <center> CONSULTAR LA VERSION DE GIT</center>

In [7]:
comando = 'git --version'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git --version
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
git version 2.25.0.windows.1
##############################################################################################################



# <center> CLONAR UN REPOSITORIO GIT</center>

In [8]:
repositorio_base = 'https://github.com/JorgeCardona'
nombre_del_repositorio = 'comandos_git'

eliminar_directorio(nombre_repositorio=nombre_del_repositorio)

comando = 'git clone {repositorio_base}/{nombre_del_repositorio}.git'.format(repositorio_base=repositorio_base, nombre_del_repositorio=nombre_del_repositorio)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git clone https://github.com/JorgeCardona/comandos_git.git
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



## FETCH
<center>
    <img width="75%" align="center" src="fecth.gif">
</center>

## PULL
<center>
    <img width="75%" align="center" src="pull.gif">
</center>

# <center> CAMBIAR A LA CARPETA PARA TRABAJAR CON EL REPOSITORIO</center>

In [9]:
#importing the os module
import os

#to get the current working directory
directory = os.getcwd()
directory

'C:\\z Github\\Git Comandos'

In [10]:
os.chdir('{directory}/{repository}'.format(directory=directory, repository=nombre_del_repositorio))
directory = os.getcwd()
directory

'C:\\z Github\\Git Comandos\\comandos_git'

# <center> MOSTRAR INFORMACION DEL REPOSITORIO</center>

In [11]:
comando = 'git remote show origin'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git remote show origin
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* remote origin
  Fetch URL: https://github.com/JorgeCardona/comandos_git.git
  Push  URL: https://github.com/JorgeCardona/comandos_git.git
  HEAD branch: main
  Remote branches:
    main          tracked
    multi_commits tracked
  Local branch configured for 'git pull':
    main merges with remote main
  Local ref configured for 'git push':
    main pushes to main (up to date)
##############################################################################################################



# <center> LISTAR LAS RAMAS DE UN REPOSITORIO </center>

### LISTA TODAS LAS RAMAS LOCALES Y LA RAMA PRINCIPAL DEL REPOSITORIO

In [12]:
comando = 'git branch'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
##############################################################################################################



### LISTA TODAS LAS RAMAS REMOTAS DEL REPOSITORIO

In [13]:
comando = 'git branch -r'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch -r
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  origin/HEAD -> origin/main
  origin/main
  origin/multi_commits
##############################################################################################################



### LISTA TODAS LAS RAMAS LOCALES INCLUYENDO LAS RAMAS OCULTAS

In [14]:
comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
  remotes/origin/multi_commits
##############################################################################################################



# <center> CREAR UNA RAMA BASADA EN LA RAMA ACTUAL DE UN REPOSITORIO</center>

In [15]:
comando = 'git branch nueva_rama'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git branch nueva_rama
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  nueva_rama
##############################################################################################################



# <center> CAMBIAR A OTRA RAMA DEL REPOSITORIO</center>

In [16]:
comando = 'git checkout nueva_rama'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout nueva_rama
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* nueva_rama
##############################################################################################################



# <center> CREAR UNA RAMA BASADA EN LA ACTUAL Y CAMBIA A LA RAMA CREADA AL MISMO TIEMPO</center>

In [17]:
comando = 'git checkout -b rama_acceso_rapido'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout -b rama_acceso_rapido
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  nueva_rama
* rama_acceso_rapido
##############################################################################################################



# <center> CREAR NUEVA RAMA LOCAL DE UNA RAMA REMOTA</center>

In [18]:
nombre_rama_remota = 'multi_commits'
copia_rama_remota = '{nombre_rama_remota}_copia'.format(nombre_rama_remota=nombre_rama_remota)
comando = 'git checkout -b {copia_rama_remota} origin/{nombre_rama_remota}'.format(copia_rama_remota=copia_rama_remota, nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout -b multi_commits_copia origin/multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Branch 'multi_commits_copia' set up to track remote branch 'multi_commits' from 'origin'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* multi_commits_copia
  nueva_rama
  rama_acceso_rapido
##############################################################################################################



# <center> DESCARGAR RAMA REMOTA A LOCAL Y CAMBIAR A LA RAMA BAJADA AL MISMO TIEMPO</center>

In [19]:
comando = 'git checkout -t origin/{nombre_rama_remota}'.format(nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout -t origin/multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Branch 'multi_commits' set up to track remote branch 'multi_commits' from 'origin'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
* multi_commits
  multi_commits_copia
  nueva_rama
  rama_acceso_rapido
##############################################################################################################



# <center> RENOMBRAR UNA RAMA DEL REPOSITORIO</center>

In [20]:
comando = 'git checkout {copia_rama_remota}'.format(copia_rama_remota=copia_rama_remota)
ejecutar_comando_git(comando)

comando = 'git branch -m nueva_rama nueva_rama_renombrada'
ejecutar_comando_git(comando)

# Alternativa
comando = 'git branch --move rama_acceso_rapido rama_acceso_rapido_renombrada'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout multi_commits_copia
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/multi_commits'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -m nueva_rama nueva_rama_renombrada
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch --move rama_acceso_rapido rama_acceso_rapido_renombrada
###################

# <center> OBTENER LA DIFERENCIA EN CONTENIDO ENTRE 2 RAMAS COMMITs DE UN REPOSITORIO</center>

In [21]:
# confirmar al repositorio remoto el reset
comando = 'git diff main multi_commits'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git diff main multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
diff --git a/archivo.csv b/archivo.csv
new file mode 100644
index 0000000..e69de29
diff --git a/archivo.pdf b/archivo.pdf
new file mode 100644
index 0000000..31a8a6a
--- /dev/null
+++ b/archivo.pdf
@@ -0,0 +1 @@
+
\ No newline at end of file
diff --git a/archivo.xlsx b/archivo.xlsx
new file mode 100644
index 0000000..53f0f1e
Binary files /dev/null and b/archivo.xlsx differ
diff --git a/index.html b/index.html
new file mode 100644
index 0000000..e69de29
##############################################################################################################



# <center> RESTAURAR O RECUPERAR ARCHIVOS MODIFICADOS O ELIMINADOS DE UN REPOSITORIO GIT <br> SOLO MODIFICACIONES EN REPOSITORIO LOCAL, BASADO EN EL ULTIMO COMMIT <br/></center>

In [22]:
nombre_rama_remota = 'main'
generar_archivos()

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1679765918.271976 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1679765918.384136 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #######

['info.txt',
 'Dockerfile',
 'Manifest.yaml',
 'imagen.png',
 'main.py',
 'connection.java',
 'calculos.c',
 'addon.php',
 'utils.gif',
 'query.sql',
 'dataset.avro',
 'link.vs']

In [23]:
eliminar_archivos_directorio()

"Eliminacion Completada con Exito!!! de ['addon.php', 'archivo.csv', 'archivo.pdf', 'archivo.xlsx', 'calculos.c', 'config.json', 'connection.java', 'dataset.avro', 'Dockerfile', 'imagen.png', 'index.html', 'info.txt', 'LICENSE', 'link.vs', 'main.py', 'Manifest.yaml', 'query.sql', 'README.md', 'requirements.txt', 'utils.gif']"

### RESTAURAR UN ARCHIVO ESPECIFICO

In [24]:
comando = 'git restore LICENSE'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git restore LICENSE
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



### RESTAURAR TODOS LOS ARCHIVOS AL MISMO TIEMPO

In [25]:
comando = 'git restore .'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



# <center> VERIFICAR EL ESTADO DE LOS ARCHIVOS EN UN REPOSITORIO GIT</center>

In [26]:
generar_archivos()

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1679765923.054338 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1679765923.174274 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #######

['info.txt',
 'Dockerfile',
 'Manifest.yaml',
 'imagen.png',
 'main.py',
 'connection.java',
 'calculos.c',
 'addon.php',
 'utils.gif',
 'query.sql',
 'dataset.avro',
 'link.vs']

In [27]:
comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch multi_commits_copia
Your branch is up to date with 'origin/multi_commits'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	info.txt
	link.vs
	main.py
	query.sql
	utils.gif

nothing added to commit but untracked files present (use "git add" to track)
##############################################################################################################



# <center> AGREGAR CAMBIOS DE ARCHIVOS A UN REPOSITORIO GIT</center>

### AGREGAR EL CAMBIO DE UN ARCHIVO O UN SOLO ARCHIVO NUEVO

In [28]:
comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch multi_commits_copia
Your branch is up to date with 'origin/multi_commits'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   info.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	link.vs
	main.py
	query.sql
	utils.gif

##############

### AGREGAR LOS CAMBIOS DE TODOS LOS ARCHIVOS O TODOS LOS CAMBIOS Y ARCHIVOS NUEVOS

In [29]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git add .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch multi_commits_copia
Your branch is up to date with 'origin/multi_commits'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   Dockerfile
	new file:   Manifest.yaml
	new file:   addon.php
	new file:   calculos.c
	new file:   connection.java
	new file:   dataset.avro
	new file:   imagen.png
	new file:   info.txt
	new file:   link.vs
	new file:   main.py
	new file:   query.

# <center> HACER COMMIT A UN REPOSITORIO GIT</center>

In [30]:
# solo en este paso, al hacer el primer commit se crea la rama
comando = 'git commit -m "adicionados archivo de prueba"'
ejecutar_comando_git(comando)
        
comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git commit -m "adicionados archivo de prueba"
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
[multi_commits_copia c3d4a0b] adicionados archivo de prueba
 12 files changed, 12 insertions(+)
 create mode 100644 Dockerfile
 create mode 100644 Manifest.yaml
 create mode 100644 addon.php
 create mode 100644 calculos.c
 create mode 100644 connection.java
 create mode 100644 dataset.avro
 create mode 100644 imagen.png
 create mode 100644 info.txt
 create mode 100644 link.vs
 create mode 100644 main.py
 create mode 100644 query.sql
 create mode 100644 utils.gif
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
###############################

# <center> PUBLICAR UNA RAMA LOCAL QUE NO EXISTE EN REMOTO A UN REPOSITORIO GIT <br>https://github.com/JorgeCardona/comandos_git</br> </center>


In [31]:
comando = 'git push --set-upstream origin {copia_rama_remota}'.format(copia_rama_remota=copia_rama_remota)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git push --set-upstream origin multi_commits_copia
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Branch 'multi_commits_copia' set up to track remote branch 'multi_commits_copia' from 'origin'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  multi_commits
* multi_commits_copia
  nueva_rama_renombrada
  rama_acceso_rapido_renombrada
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
  remotes/origin/multi_commits
  remotes/origin/multi_commits_copia
#######################################################

# <center> ADICIONAR ARCHIVOS NUEVOS A LA RAMA</center>

In [32]:
generar_archivos()

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1679765933.971076 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1679765934.095449 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #######

['info.txt',
 'Dockerfile',
 'Manifest.yaml',
 'imagen.png',
 'main.py',
 'connection.java',
 'calculos.c',
 'addon.php',
 'utils.gif',
 'query.sql',
 'dataset.avro',
 'link.vs']

# SUBIR NUEVOS ARCHIVOS A UNA RAMA DEL REPOSITORIO  <br>https://github.com/JorgeCardona/comandos_git</br> 

In [33]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionados archivos nuevos"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git add .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git commit -m "adicionados archivos nuevos"
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
[multi_commits_copia 08fd474] adicionados archivos nuevos
 12 files changed, 12 insertions(+)
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO E

In [34]:
# siempre origin para repositorios remotos
comando = 'git push origin'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git push origin
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



# <center> ELIMINAR UNA RAMA DE UN REPOSITORIO GIT</center>

## ELIMINAR RAMA DE UN REPOSITORIO LOCAL

In [35]:
# no se puede eliminar una rama si se esta trabajando en ese monento en ella
comando = 'git checkout main'
ejecutar_comando_git(comando)

ramas_a_eliminar = ['nueva_rama_renombrada','rama_acceso_rapido_renombrada',copia_rama_remota]

# eliminar lista de ramas
for rama in ramas_a_eliminar:
    # -D es diferenete de -d, con mayusculas elimina la rama asi existan commits por confirmar
    comando = 'git branch -D {rama}'.format(rama=rama)
    ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D nueva_rama_renombrada
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Deleted branch nueva_rama_renombrada (was 3b4edb5).
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D rama_acceso_rapido_renombrada
#########################

## ELIMINAR RAMA DE UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br> 

In [36]:
comando = 'git push origin -d {copia_rama_remota}'.format(copia_rama_remota=copia_rama_remota)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git push origin -d multi_commits_copia
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  multi_commits
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
  remotes/origin/multi_commits
##############################################################################################################



# <center> CREAR RAMA VACIA QUE NO ESTE RELACIONADA CON NINGUN CONTENIDO DE OTRA RAMA</center>
<center>
    <img width="100%" align="center" src="ciclo_de_vida_archuvo_git.png">
</center>

### UNTRACKED - ES UN FICHERO NUEVO QUE SE HA CREADO Y GIT TODAVIA NO TIENE CONSTANCIA, NO HACE PARTE DE NINGUN COMMIT ANTERIOR
### TRACKED - UNSTAGE -> FICHERO QUE HACE PARTE DE COMMITS ANTERIORES, HA SIDO MODIFICADO PERO TODAVIA NO SE HA ENVIADO AL AREA DE STAGING
### TRACKED - STAGE -> SE REALIZO UN CAMBIO Y SE HA ENVIADO A LA ZONA STAGING (git add archivo)
### TRACKED - COMMITED -> UN ARCHIVO QUE SE HA HECHO COMMIT Y QUE YA ESTA EN LA ZONA EL REPOSITORIO LOCAL (git commit -m 'archivo adicionado al repo local')

### CREA LA RAMA

In [37]:
rama = 'utils'
comando = 'git switch --orphan {rama}'.format(rama=rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git switch --orphan utils
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  main
  multi_commits
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
  remotes/origin/multi_commits
##############################################################################################################



### CREA ARCHIVOS EN EL WORKING DIRECTORY

In [38]:
generar_archivos()

comando = 'git status'
ejecutar_comando_git(comando)

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1679765942.259143 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1679765942.385781 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #######

### ADICIONA LOS ARCHIVOS AL STAGE

#### ADICIONA UN SOLO ARCHIVO AL STAGE

In [39]:
comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   info.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	link.vs
	main.py
	query.sql
	utils.gif

########################################################################

#### ADICIONA TODOS LOS ARCHIVOS AL STAGE

In [40]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   Dockerfile
	new file:   Manifest.yaml
	new file:   addon.php
	new file:   calculos.c
	new file:   connection.java
	new file:   dataset.avro
	new file:   imagen.png
	new file:   info.txt
	new file:   link.vs
	new file:   main.py
	new file:   query.sql
	new file:   utils.gif

##############################

### ELIMINA LOS ARCHIVOS DEL STAGE Y LOS RETORNA AL WORKING DIRECTORY

#### ELIMINA UNO DE LOS ARCHIVOS DEL STAGE

In [41]:
comando = 'git rm --cached info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git rm --cached info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
rm 'info.txt'
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   Dockerfile
	new file:   Manifest.yaml
	new file:   addon.php
	new file:   calculos.c
	new file:   connection.java
	new file:   dataset.avro
	new file:   imagen.png
	new file:   link.vs
	new file:   main.py
	new file:   query.sql
	new file:   utils.gif

Untracked files:
  (use

#### ELIMINA TODOS DE LOS ARCHIVOS DEL STAGE

In [42]:
comando = 'git rm --cached "*"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git rm --cached "*"
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
rm 'Dockerfile'
rm 'Manifest.yaml'
rm 'addon.php'
rm 'calculos.c'
rm 'connection.java'
rm 'dataset.avro'
rm 'imagen.png'
rm 'link.vs'
rm 'main.py'
rm 'query.sql'
rm 'utils.gif'
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	info.txt
	link.vs
	main.py


## ELIMINA TODOS LOS ARCHIVOS UNTRACKED DEL WORKING DIRECTORY

#### ADICIONA ARCHIVOS Y CREA EL COMMIT

In [43]:
comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionado archivo INFO.txt"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git add info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   info.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dockerfile
	Manifest.yaml
	addon.php
	calculos.c
	connection.java
	dataset.avro
	imagen.png
	link.vs
	main.py
	query.sql
	utils.gif

########################################################################

#### ELIMINA TODOS LOS ARCHIVOS UNTRACKED DEL WORKING DIRECTORY

In [44]:
comando = 'git clean -f'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git clean -f
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Removing Dockerfile
Removing Manifest.yaml
Removing addon.php
Removing calculos.c
Removing connection.java
Removing dataset.avro
Removing imagen.png
Removing link.vs
Removing main.py
Removing query.sql
Removing utils.gif
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils
nothing to commit, working tree clean
##############################################################################################################



### ADICIONA LOS ARCHIVOS AL STAGE Y HACE LO ALMACENA EN EL REPOSITORIO LOCAL CON UN COMMIT

In [45]:
generar_archivos()

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionados archivos de prueba"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1679765949.367011 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1679765949.490369 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #######

### DESHACER UN COMMIT LOCAL
### ELIMINA LOS ARCHIVOS DEL LOCAL REPOSITORY Y LOS RETORNA AL STAGING AREA

In [46]:
crear_commits(total_commits=5)

lista_de_archivos = ['info.txt', 'Dockerfile', 'Manifest.yaml', 'imagen.png', 'main.py', 'connection.java', 'calculos.c', 'addon.php', 'utils.gif', 'query.sql', 'dataset.avro', 'link.vs']
############################################# COMANDO A EJECUTAR #############################################
echo 1679765951.469989 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
echo 1679765951.585798 >> Dockerfile
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #######

In [48]:
# define el commit al que se quiere regresar
commit_id= '5b478a2'

comando = 'git reset {commit_id}'.format(commit_id=commit_id)
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

# eliminar los archivos que no estan en el local repo, para evitar errores al cambiar de rama
comando = 'git rm --cached "*"'
ejecutar_comando_git(comando)

comando = 'git clean -f'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git reset 5b478a2
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Unstaged changes after reset:
M	Dockerfile
M	Manifest.yaml
M	addon.php
M	calculos.c
M	connection.java
M	dataset.avro
M	imagen.png
M	info.txt
M	link.vs
M	main.py
M	query.sql
M	utils.gif
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch utils
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Dockerfile
	modified:   Manifest.yaml

In [49]:
# crea un nuevo commit para usarlo luego en el reflog
generar_archivos(todos_los_archivos=False)

comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionado archivo INFO.txt PARA USAR EN REFLOG"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

lista_de_archivos = ['info.txt']
############################################# COMANDO A EJECUTAR #############################################
echo 1679766004.069185 >> info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git add info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch util

## ELIMINAR LA RAMA

In [50]:
# no se puede eliminar una rama si se esta trabajando en ese monento en ella
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git branch -D {rama}'.format(rama=rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D utils
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Deleted branch utils (was 39954ce).
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO ################

# <center> LISTAR EL HISTORIAL DE COMMITS DE UN REPOSITORIO GIT</center>

### VER EL HISTORIAL COMPLETO DE LA RAMA MAIN

In [51]:
comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 3b4edb5a785dcc5841b35cd2ad93033cd29cb730
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Fri Mar 17 20:39:55 2023 -0500

    adicionados archivo de configuracion y requerimientos

commit d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
Author: JorgeCardona <jorgecardona@utp.edu.co>
Date:   Fri Mar 17 20:27:49 2023 -0500

    Initial commit
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  multi_commits
################################################

### VER HISTORIAL OTRA RAMA REMOTA

In [52]:
nombre_rama = 'commit_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* main
  multi_com

In [53]:
comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 0c39f4b9bf83965814ccb7539641b149ee528a65
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:18 2023 -0500

    ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618

commit 7e00344b0dedbe9113c6241e773cab0bc23a9784
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618

commit f90c5ed48f9b0bd52a86d26dbafd662ac9ffaf08
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618

commit d782f76fc92870d53547e03f942032221b5f7911
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO

### VER HISTORIAL DE LOS ULTIMOS n COMMITS

In [54]:
comando = 'git log -3' 
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log -3
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 0c39f4b9bf83965814ccb7539641b149ee528a65
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:18 2023 -0500

    ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618

commit 7e00344b0dedbe9113c6241e773cab0bc23a9784
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618

commit f90c5ed48f9b0bd52a86d26dbafd662ac9ffaf08
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
##############################################################################################################

############################################# COM

### VER HISTORIAL RESUMIDO DE UNA RAMA

In [55]:
comando = 'git log --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
0c39f4b ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
f90c5ed ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
d782f76 ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:14.762618
2fe07ec ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:14.762618
836604d ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:14.762618
191c29a ADICIONADO ARCHIVO connection.java 2023-03-25 12:40:14.762618
2cb864d ADICIONADO ARCHIVO main.py 2023-03-25 12:40:14.762618
d8268f2 ADICIONADO ARCHIVO imagen.png 2023-03-25 12:40:14.762618
9ddf7d4 ADICIONADO ARCHIVO Manifest.yaml 2023-03-25 12:40:14.762618
6a0dded ADICIONADO ARCHIVO Dockerfile 2023-03-25 12:40:14.762618
eaf55b1 ADICIONADO ARCHIVO info.txt 2023-03-

### VER HISTORIAL RESUMIDO DE UNA RAMA DE n COMMITS

In [56]:
comando = 'git log --oneline -3'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline -3
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
0c39f4b ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
f90c5ed ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* commit_branch
  main
  multi_commits
##############################################################################################################



## VER HISTORIAL RESUMIDO DE UNA RAMA SIN ESTAR EN LA RAMA

In [57]:
comando = 'git log --oneline main'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
3b4edb5 adicionados archivo de configuracion y requerimientos
d15fad2 Initial commit
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* commit_branch
  main
  multi_commits
##############################################################################################################



### VER HISTORIAL DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS

In [58]:
comando = 'git log -2 -p'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log -2 -p
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 0c39f4b9bf83965814ccb7539641b149ee528a65
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:18 2023 -0500

    ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618

diff --git a/link.vs b/link.vs
new file mode 100644
index 0000000..76935ad
--- /dev/null
+++ b/link.vs
@@ -0,0 +1 @@
+1679766016.29302 

commit 7e00344b0dedbe9113c6241e773cab0bc23a9784
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618

diff --git a/dataset.avro b/dataset.avro
new file mode 100644
index 0000000..d8cf289
--- /dev/null
+++ b/dataset.avro
@@ -0,0 +1 @@
+1679766016.148312 
######################################################################

### VER HISTORIAL DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS USANDO **SHOW**

In [59]:
comando = 'git show -2'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git show -2
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 0c39f4b9bf83965814ccb7539641b149ee528a65
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:18 2023 -0500

    ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618

diff --git a/link.vs b/link.vs
new file mode 100644
index 0000000..76935ad
--- /dev/null
+++ b/link.vs
@@ -0,0 +1 @@
+1679766016.29302 

commit 7e00344b0dedbe9113c6241e773cab0bc23a9784
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:17 2023 -0500

    ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618

diff --git a/dataset.avro b/dataset.avro
new file mode 100644
index 0000000..d8cf289
--- /dev/null
+++ b/dataset.avro
@@ -0,0 +1 @@
+1679766016.148312 
########################################################################

### VER HISTORIAL RESUMIDO DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS

In [60]:
comando = 'git log --oneline -2 -p'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline -2 -p
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
0c39f4b ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
diff --git a/link.vs b/link.vs
new file mode 100644
index 0000000..76935ad
--- /dev/null
+++ b/link.vs
@@ -0,0 +1 @@
+1679766016.29302 
7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
diff --git a/dataset.avro b/dataset.avro
new file mode 100644
index 0000000..d8cf289
--- /dev/null
+++ b/dataset.avro
@@ -0,0 +1 @@
+1679766016.148312 
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO ################################

### VER HISTORIAL DE n COMMITS ANTES DE UNA FECHA ESPECIFICA

In [61]:
comando = 'git log --oneline --before={2019-05-30}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline --before={2019-05-30}
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* commit_branch
  main
  multi_commits
##############################################################################################################



### VER HISTORIAL DE n COMMITS DESPUES DE UNA FECHA ESPECIFICA

In [62]:
comando = 'git log --oneline --after={2019-05-30}'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline --after={2019-05-30}
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
0c39f4b ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
f90c5ed ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
d782f76 ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:14.762618
2fe07ec ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:14.762618
836604d ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:14.762618
191c29a ADICIONADO ARCHIVO connection.java 2023-03-25 12:40:14.762618
2cb864d ADICIONADO ARCHIVO main.py 2023-03-25 12:40:14.762618
d8268f2 ADICIONADO ARCHIVO imagen.png 2023-03-25 12:40:14.762618
9ddf7d4 ADICIONADO ARCHIVO Manifest.yaml 2023-03-25 12:40:14.762618
6a0dded ADICIONADO ARCHIVO Dockerfile 2023-03-25 12:40:14.762618
eaf55b1 ADICIONADO ARCH

### VER HISTORIAL DE QUIEN HA MANIPULADO UN ARCHIVO ESPECIFICO

In [63]:
comando = 'git blame requirements.txt'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git blame requirements.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* commit_branch
  main
  multi_commits
##############################################################################################################



### VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO

In [64]:
comando = 'git log --graph --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --graph --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* 0c39f4b ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
* 7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
* f90c5ed ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
* d782f76 ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:14.762618
* 2fe07ec ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:14.762618
* 836604d ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:14.762618
* 191c29a ADICIONADO ARCHIVO connection.java 2023-03-25 12:40:14.762618
* 2cb864d ADICIONADO ARCHIVO main.py 2023-03-25 12:40:14.762618
* d8268f2 ADICIONADO ARCHIVO imagen.png 2023-03-25 12:40:14.762618
* 9ddf7d4 ADICIONADO ARCHIVO Manifest.yaml 2023-03-25 12:40:14.762618
* 6a0dded ADICIONADO ARCHIVO Dockerfile 2023-03-25 12:40:14.762618
* eaf55b1 ADIC

### VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO MAS DETALLADO

In [65]:
comando = 'git log --decorate --graph --oneline --all'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --decorate --graph --oneline --all
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* 0c39f4b (HEAD -> commit_branch, origin/commit_branch) ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
* 7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
* f90c5ed ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
* d782f76 ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:14.762618
* 2fe07ec ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:14.762618
* 836604d ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:14.762618
* 191c29a ADICIONADO ARCHIVO connection.java 2023-03-25 12:40:14.762618
* 2cb864d ADICIONADO ARCHIVO main.py 2023-03-25 12:40:14.762618
* d8268f2 ADICIONADO ARCHIVO imagen.png 2023-03-25 12:40:14.762618
* 9ddf7d4 ADICIONADO ARCHIVO Manifest.yaml 2023-03-25 12:40:14.762618
* 6a0dded ADICIONA

# <center> MOSTRAR LOS METADATOS Y LOS CAMBIOS DE UN COMMIT ESPECIFICO</center>

In [66]:
# solo en este paso, al hacer el primer commit se crea la rama
comando = 'git show d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48'
ejecutar_comando_git(comando)
        
comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git show d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
Author: JorgeCardona <jorgecardona@utp.edu.co>
Date:   Fri Mar 17 20:27:49 2023 -0500

    Initial commit

diff --git a/.gitignore b/.gitignore
new file mode 100644
index 0000000..b6e4761
--- /dev/null
+++ b/.gitignore
@@ -0,0 +1,129 @@
+# Byte-compiled / optimized / DLL files
+__pycache__/
+*.py[cod]
+*$py.class
+
+# C extensions
+*.so
+
+# Distribution / packaging
+.Python
+build/
+develop-eggs/
+dist/
+downloads/
+eggs/
+.eggs/
+lib/
+lib64/
+parts/
+sdist/
+var/
+wheels/
+pip-wheel-metadata/
+share/python-wheels/
+*.egg-info/
+.installed.cfg
+*.egg
+MANIFEST
+
+# PyInstaller
+#  Usually these files are written by a python script from a template
+#  before PyInstaller 

# <center> BUSCAR FRASE DENTRO DE TODOS LOS ARCHIVOS DE LA RAMA DEL REPOSITORIO</center>

### BUSQUEDA SENSIBLE AL CASO

In [67]:
frase_a_buscar = '7'

# la busqueda es sensible al caso
comando = 'git grep {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git grep 7
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Dockerfile:1679766014.98642 
LICENSE:                       Version 3, 29 June 2007
LICENSE: Copyright (C) 2007 Free Software Foundation, Inc. <https://fsf.org/>
LICENSE:non-permissive terms added in accord with section 7 apply to the code;
LICENSE:    7.  This requirement modifies the requirement in section 4 to
LICENSE:    License will therefore apply, along with any applicable section 7
LICENSE:  7. Additional Terms.
LICENSE:or that patent license was granted, prior to 28 March 2007.
LICENSE:  17. Interpretation of Sections 15 and 16.
Manifest.yaml:1679766015.152563 
addon.php:1679766015.781202 
calculos.c:1679766015.664656 
connection.java:1679766015.544858 
dataset.avro:1679766016.148312 
imagen.png:1679766015.286207 
info.txt:1679766014.870386 
link.v

### BUSQUEDA NO SENSIBLE AL CASO

In [68]:
# busca mayusculas y minusculas
comando = 'git grep -i {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git grep -i 7
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Dockerfile:1679766014.98642 
LICENSE:                       Version 3, 29 June 2007
LICENSE: Copyright (C) 2007 Free Software Foundation, Inc. <https://fsf.org/>
LICENSE:non-permissive terms added in accord with section 7 apply to the code;
LICENSE:    7.  This requirement modifies the requirement in section 4 to
LICENSE:    License will therefore apply, along with any applicable section 7
LICENSE:  7. Additional Terms.
LICENSE:or that patent license was granted, prior to 28 March 2007.
LICENSE:  17. Interpretation of Sections 15 and 16.
Manifest.yaml:1679766015.152563 
addon.php:1679766015.781202 
calculos.c:1679766015.664656 
connection.java:1679766015.544858 
dataset.avro:1679766016.148312 
imagen.png:1679766015.286207 
info.txt:1679766014.870386 
lin

### BUSQUEDA POR PALABRA ESPECIFICA

In [69]:
# busca la palabra sola sin mezcla, que no sea prefijo o sufijo, los simbolos no bloquean la busqueda
comando = 'git grep -iw {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git grep -iw 7
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
LICENSE:non-permissive terms added in accord with section 7 apply to the code;
LICENSE:    7.  This requirement modifies the requirement in section 4 to
LICENSE:    License will therefore apply, along with any applicable section 7
LICENSE:  7. Additional Terms.
##############################################################################################################



### BUSQUEDA CON INTERVALO DE LINEAS POR ENCIMA Y POR DEBAJO DEL TEXTO A BUSCAR

In [70]:
# divide el texto y muestra con C cuantas lineas detexto por debajo y por encima existe al filtrar la palabra deseada
comando = 'git grep -i -C 2 {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git grep -i -C 2 7
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Dockerfile:1679766014.98642 
--
LICENSE-                    GNU GENERAL PUBLIC LICENSE
LICENSE:                       Version 3, 29 June 2007
LICENSE-
LICENSE: Copyright (C) 2007 Free Software Foundation, Inc. <https://fsf.org/>
LICENSE- Everyone is permitted to copy and distribute verbatim copies
LICENSE- of this license document, but changing it is not allowed.
--
LICENSE-appropriately publish on each copy an appropriate copyright notice;
LICENSE-keep intact all notices stating that this License and any
LICENSE:non-permissive terms added in accord with section 7 apply to the code;
LICENSE-keep intact all notices of the absence of any warranty; and give all
LICENSE-recipients a copy of this License along with the Program.
--
LICENSE-    b) The work

### MOSTRAR EL ARCHIVO Y EL NUMERO DE LA LINEA DONDE ESTA EL TEXTO BUSCADO

In [71]:
comando = 'git grep -i -n {frase_a_buscar}'.format(frase_a_buscar=frase_a_buscar)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git grep -i -n 7
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Dockerfile:1:1679766014.98642 
LICENSE:2:                       Version 3, 29 June 2007
LICENSE:4: Copyright (C) 2007 Free Software Foundation, Inc. <https://fsf.org/>
LICENSE:201:non-permissive terms added in accord with section 7 apply to the code;
LICENSE:219:    7.  This requirement modifies the requirement in section 4 to
LICENSE:224:    License will therefore apply, along with any applicable section 7
LICENSE:343:  7. Additional Terms.
LICENSE:534:or that patent license was granted, prior to 28 March 2007.
LICENSE:612:  17. Interpretation of Sections 15 and 16.
Manifest.yaml:1:1679766015.152563 
addon.php:1:1679766015.781202 
calculos.c:1:1679766015.664656 
connection.java:1:1679766015.544858 
dataset.avro:1:1679766016.148312 
imagen.png:1:16797

# <center> MODIFICAR EL MENSAJE DEL ULTIMO COMMIT</center>

In [72]:
nombre_rama = 'amend_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama, todos_los_archivos=False)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  commit_branch
* 

# EN LA CONSOLA DE VSCODE
# git commit --amend

## SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar el emnsaje del commit anterior por el que se quiere y cerrar el archivo
## validar en local con git log, no afecta la hora del commit original con la fecha del mensaje actualizado
## subir los cambios al repositorio¶
# git push --force origin amend_branch

In [73]:
comando = 'git log'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git log
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit a0c410879e050a7204242ba65e564ef858bfad12
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:29 2023 -0500

    ADICIONADO ARCHIVO info.txt 2023-03-25 12:40:28.804483

commit 3b4edb5a785dcc5841b35cd2ad93033cd29cb730
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Fri Mar 17 20:39:55 2023 -0500

    adicionados archivo de configuracion y requerimientos

commit d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
Author: JorgeCardona <jorgecardona@utp.edu.co>
Date:   Fri Mar 17 20:27:49 2023 -0500

    Initial commit
##############################################################################################################



#### ELIMINAR LA RAMA EN EL REPOSITORIO REMOTO Y LOCAL

In [74]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git push origin -d amend_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D amend_branch
####################################### RESULTADO DE COMANDO EJECUTADO ###########################

# <center> REGRESAR A UNA VERSION ANTERIOR DE UN REPOSITORIO GIT</center>

### DETACHED - MOVERSE A UN COMMIT ESPECIFICO Y PONER ESTE COMMIT CON LA NUEVA HEAD

In [75]:
comando = 'git checkout multi_commits'
ejecutar_comando_git(comando)

comando = 'git checkout d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git checkout multi_commits
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/multi_commits'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUT

### RETORNA AL HEAD ORIGINAL DE LA RAMA

In [76]:
comando = 'git switch -'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git switch -
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/multi_commits'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
67e732f adicionado archivo html
1d9c154 adicionado archivo csv
dcfb55b adicionado archivo excel
52ff70b adicionado archivo pdf
3b4edb5 adicionados archivo de configuracion y requerimientos
d15fad2 Initial commit
##############################################################################################################

##################################

# <center> RESET COMMITs DE UN REPOSITORIO <br> ELIMINA LOS COMMITS Y AHORA EL NUEVO HEAD ES EL COMMIT A DONDE SE QUIERE IR</br></center>

In [77]:
nombre_rama = 'reset_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  commit_branch
* 

### SOFT RESET n COMMITs **HEAD~n** MANTENIENDO LOS CAMBIOS QUE SE HABIAN HECHO EN LOS ARCHIVOS LOCALES
<center>
    <img width="75%" align="center" src="reset_soft.gif">
</center>
<br>
<center>
    <img width="75%" align="center" src="reflog_reset.gif">
</center>

In [78]:
# MANITIENE todos los archivos o cambios hechos despues de este commit Y PUEDEN SER GUARDADOS DE NUEVO
comando = 'git reset --soft HEAD~3'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git reset --soft HEAD~3
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
c6dea65 ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:35.588307
422cdc0 ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:35.588307
fe1c341 ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:35.588307
cb1b8cd ADICIONADO ARCHIVO connection.java 2023-03-25 12:40:35.588307
eb19244 ADICIONADO ARCHIVO main.py 2023-03-25 12:40:35.588307
e69fb7d ADICIONADO ARCHIVO imagen.png 2023-03-25 12:40:35.588307
5c31166 ADIC

### HARD RESET n COMMITs **HEAD~n** ELIMINANDO LOS ARCHIVOS Y CAMBIOS QUE SE HABIAN HECHO HASTA ESTE COMMIT
<center>
    <img width="75%" align="center" src="reset_hard.gif">
</center>

In [79]:
# elimina todos los archivos o cambios hechos despues de este commit YA NO SE PUEDEN RECUPERAR ESOS ARCHIVOS
comando = 'git reset --hard HEAD~7'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git reset --hard HEAD~7
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
HEAD is now at 89ca6e3 ADICIONADO ARCHIVO Dockerfile 2023-03-25 12:40:35.588307
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
89ca6e3 ADICIONADO ARCHIVO Dockerfile 2023-03-25 12:40:35.588307
8f20696 ADICIONADO ARCHIVO info.txt 2023-03-25 12:40:35.588307
3b4edb5 adicionados archivo de configuracion y requerimientos
d15fad2 Initial commit
#############################################################################################################

### RESET COMMITs ELIMINANDO LOS ARCHIVOS Y CAMBIOS HASTA UN COMMIT ESPECIFICO USANDO EL ID DEL COMMIT

In [80]:
comando = 'git reset --hard 3b4edb5a785dcc5841b35cd2ad93033cd29cb730'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git reset --hard 3b4edb5a785dcc5841b35cd2ad93033cd29cb730
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
HEAD is now at 3b4edb5 adicionados archivo de configuracion y requerimientos
##############################################################################################################



## RESET HASTA UN COMMIT ESPECIFICO EN UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [81]:
comando = 'git reset --hard d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git reset --hard d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
HEAD is now at d15fad2 Initial commit
##############################################################################################################



#### CONFIRMAR LOS CAMBIOS DEL RESET EN EL REPOSITORIO DE ORIGEN O REMOTO

In [82]:
# confirmar los cambios al repositorio remoto el reset
comando = 'git push --force origin {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git push --force origin reset_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



#### LISTAR HISTORIAL DE CAMBIOS HECHOS EN EL REPOSITORIO LOCAL

In [83]:
comando = 'git reflog'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git reflog
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
d15fad2 HEAD@{0}: reset: moving to d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48
3b4edb5 HEAD@{1}: reset: moving to 3b4edb5a785dcc5841b35cd2ad93033cd29cb730
89ca6e3 HEAD@{2}: reset: moving to HEAD~7
c6dea65 HEAD@{3}: reset: moving to HEAD~3
c2ec951 HEAD@{4}: commit: ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:35.588307
ff5f700 HEAD@{5}: commit: ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:35.588307
6d55e37 HEAD@{6}: commit: ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:35.588307
c6dea65 HEAD@{7}: commit: ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:35.588307
422cdc0 HEAD@{8}: commit: ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:35.588307
fe1c341 HEAD@{9}: commit: ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:35.588307
cb1b8cd HEAD@{10}: commit: ADICIONADO

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [84]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git push origin -d reset_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D reset_branch
####################################### RESULTADO DE COMANDO EJECUTADO ###########################

# <center> REVERTIR COMMITs DE UN REPOSITORIO <br> CREA UN NUEVO COMMIT MANTENIENDO LOS COMMITS ANTERIORES</br></center>
<center>
    <img width="75%" align="center" src="revert.gif">
</center>

In [85]:
nombre_rama = 'revert_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  commit_branch
* 

## REVERT HASTA UN COMMIT ESPECIFICO O UN HEAD~ EN UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [86]:
# elimina todos los archivos o cambios hechos despues de este commit YA NO SE PUEDEN RECUPERAR ESOS ARCHIVOS
comando = 'git revert HEAD~12'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git revert HEAD~12
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Removing requirements.txt
Removing config.json
[revert_branch cc2b07f] Revert "adicionados archivo de configuracion y requerimientos"
 Date: Sat Mar 25 12:41:02 2023 -0500
 2 files changed, 0 insertions(+), 0 deletions(-)
 delete mode 100644 config.json
 delete mode 100644 requirements.txt
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
cc2b07f Revert "adicionados archivo de configuracion y requerimientos"
85f2164 ADICIONADO ARCHIVO link.vs 2023-03-25

#### CONFIRMAR LOS CAMBIOS DEL REVERT EN EL REPOSITORIO REMOTO

In [87]:
# confirmar al repositorio remoto el reset
comando = 'git push --force origin {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git push --force origin revert_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################



#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [88]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git push origin -d revert_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D revert_branch
####################################### RESULTADO DE COMANDO EJECUTADO #########################

# <center> REBASE *JUNTAR VARIOS COMMITS EN UNO SOLO, CAMBIAR EL ORDEN DE LOS COMMITS <br>REESCRIBIENDO EL HISTORIAL Y LOS MENSAJES DE MULTIPLES* COMMITs DE UN REPOSITORIO</br> <br>https://github.com/JorgeCardona/comandos_git</br></center>
<center>
    <img width="75%" align="center" src="rebase.gif">
</center>

In [89]:
nombre_rama = 'rebase_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  commit_branch
* 

# EN LA CONSOLA, TERMINAL O UN IDE DE DESARROLLO <br>https://github.com/JorgeCardona/comandos_git</br>

<center>
    <img width="75%" align="center" src="rebase_interactivo.gif">
</center>
<br>
<center>
    <img width="75%" align="center" src="rebase_tipos.png">
</center>
<br>
<center>
    <img width="75%" align="center" src="rebase_squash.gif">
</center>

### n COMMITS ATRAS
# git rebase -i HEAD~9

### UN COMMIT ESPECIFICO
# git rebase -i d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48


##### SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar la palabra **pick** por **squash** GUARDAR y CERRAR
### DEJAR EL PRIMERO COMO PICK YA QUE SINO, NO EXISTEN COMMITS PARA HACER, GENERA ERROR.
##### SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar los comentarios por el comentario que se quiere
## subir los cambios al repositorio
# git push --force origin rebase_branch

#### ELIMINAR LA RAMA DEL REPOSITORIO LOCAL Y REMOTO

In [90]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git push origin -d rebase_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D rebase_branch
####################################### RESULTADO DE COMANDO EJECUTADO #########################

# <center> MERGE RAMAS DE UN REPOSITORIO <br>https://github.com/JorgeCardona/comandos_git</br></center>

<center>
    <img width="75%" align="center" src="merge.gif">
</center>

In [91]:
nombre_ramas = ['branch_uno','branch_dos']

for rama in nombre_ramas:
    generar_archivos_y_hacer_commit(rama, todos_los_archivos = False)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  commit_branch
* 

# <center> UNIR DOS RAMAS LOCALES </center>

<center>
    <img width="75%" align="center" src="merge_conflictos.gif">
</center>

# EN LA CONSOLA, TERMINAL O UN IDE DE DESARROLLO
# git merge branch_dos branch_uno

# Accept Current Change | Accept Incoming Change | Accept Both Changes | Compare Change
# <<<<<<< HEAD
# 1679198052.206751 (Current Change)
# =======
# 1679198046.401986 
# >>>>>>> branch_uno (Incoming Change)

# <center> UNIR CAMBIOS DE LA RAMA REMOTA A LA RAMA LOCAL </center> 

# EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO
<center>
    <img width="75%" align="center" src="pull.gif">
</center>
# git pull origin branch_dos

# <center> UNIR CAMBIOS DE LA RAMA LOCAL A LA RAMA REMOTA </center> 

# EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO
### hacer un git commit y despues un git pull

# Accept Current Change | Accept Incoming Change | Accept Both Changes | Compare Change
# <<<<<<< HEAD
# 1679198052.206751 (Current Change)
# =======
# 1679198046.401986 
# >>>>>>> branch_uno (Incoming Change)

## VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO

In [92]:
comando = 'git log --graph --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --graph --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* 366b573 ADICIONADO ARCHIVO info.txt 2023-03-25 12:42:51.526707
* 3b4edb5 adicionados archivo de configuracion y requerimientos
* d15fad2 Initial commit
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* branch_dos
  branch_uno
  commit_branch
  main
  multi_commits
##############################################################################################################



## VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO MAS DETALLADO

In [93]:
comando = 'git log --decorate --graph --oneline --all'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --decorate --graph --oneline --all
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* 366b573 (HEAD -> branch_dos, origin/branch_dos) ADICIONADO ARCHIVO info.txt 2023-03-25 12:42:51.526707
| * edbd06a (origin/branch_uno, branch_uno) ADICIONADO ARCHIVO info.txt 2023-03-25 12:42:47.810797
|/  
| * 0c39f4b (origin/commit_branch, commit_branch) ADICIONADO ARCHIVO link.vs 2023-03-25 12:40:14.762618
| * 7e00344 ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:40:14.762618
| * f90c5ed ADICIONADO ARCHIVO query.sql 2023-03-25 12:40:14.762618
| * d782f76 ADICIONADO ARCHIVO utils.gif 2023-03-25 12:40:14.762618
| * 2fe07ec ADICIONADO ARCHIVO addon.php 2023-03-25 12:40:14.762618
| * 836604d ADICIONADO ARCHIVO calculos.c 2023-03-25 12:40:14.762618
| * 191c29a ADICIONADO ARCHIVO connection.java 2023-03-25 12:40:14.762618
| * 

### VER HISTORIAL DE QUIEN HA MANIPULADO UN ARCHIVO ESPECIFICO

In [94]:
comando = 'git blame info.txt'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git blame info.txt
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
366b5735 (PARADIGMA\QiDimMak 2023-03-25 12:42:51 -0500 1) 1679766171.63315 
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* branch_dos
  branch_uno
  commit_branch
  main
  multi_commits
##############################################################################################################



#### ELIMINAR LAS RAMAS EN EL REPOSITORIO DE ORIGEN O REMOTO

In [95]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

for nombre_rama in nombre_ramas:
    comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

    comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git push origin -d branch_uno
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D branch_uno
####################################### RESULTADO DE COMANDO EJECUTADO ###############################

# <center> CHERRY PICK <br>https://github.com/JorgeCardona/comandos_git</br></center>

<center>
    <img width="75%" align="center" src="cherry-pick.gif">
</center>

In [96]:
nombre_rama_inicial = 'cherry_pick_branch'
nombre_rama_final = 'hotfix_branch'

nombre_ramas = [nombre_rama_inicial,nombre_rama_final]

nombre_rama = nombre_rama_inicial
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

nombre_rama = nombre_rama_final
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama, todos_los_archivos=False)

############################################# COMANDO A EJECUTAR #############################################
git restore .
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  commit_branch
* 

## VER EL LOG DE LA RAMA A USAR COMO BASE PARA EL CHERRY_PICK

In [97]:
comando = 'git log --oneline hotfix_branch'
ejecutar_comando_git(comando)


comando = 'git log --oneline cherry_pick_branch'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --oneline hotfix_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
c9dcf21 ADICIONADO ARCHIVO info.txt 2023-03-25 12:43:09.024810
3b4edb5 adicionados archivo de configuracion y requerimientos
d15fad2 Initial commit
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline cherry_pick_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
84ef7ab ADICIONADO ARCHIVO link.vs 2023-03-25 12:43:02.603526
b4326ed ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:43:02.603526
f0d2cfd ADICIONADO ARCHIVO query.sql 2023-03-25 12:43:02.603526
197a215 ADICIONADO ARCHIVO utils.g

## CHERRY PICK A UN COMMIT DE LA RAMA **CHERRY_PICK** PARA UNIRLA CON LA RAMA ACTUAL **HOTFIX**

In [99]:
unir_commits = '6bd4dfa'
comando = f'git cherry-pick {unir_commits}' # archivo main.py
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git cherry-pick 6bd4dfa
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
[hotfix_branch 168904c] ADICIONADO ARCHIVO main.py 2023-03-25 12:43:02.603526
 Date: Sat Mar 25 12:43:04 2023 -0500
 1 file changed, 1 insertion(+)
 create mode 100644 main.py
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch hotfix_branch
Your branch is ahead of 'origin/hotfix_branch' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
#############################################################

In [100]:
comando = 'git push origin'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git push origin
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git status
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
On branch hotfix_branch
Your branch is up to date with 'origin/hotfix_branch'.

nothing to commit, working tree clean
##############################################################################################################



## VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO

In [101]:
comando = 'git log --graph --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git log --graph --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
* 168904c ADICIONADO ARCHIVO main.py 2023-03-25 12:43:02.603526
* c9dcf21 ADICIONADO ARCHIVO info.txt 2023-03-25 12:43:09.024810
* 3b4edb5 adicionados archivo de configuracion y requerimientos
* d15fad2 Initial commit
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
  cherry_pick_branch
  commit_branch
* hotfix_branch
  main
  multi_commits
##############################################################################################################



#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [102]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

for nombre_rama in nombre_ramas:
    comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

    comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git push origin -d cherry_pick_branch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D cherry_pick_branch
####################################### RESULTADO DE COMANDO EJECUTADO ###############

# <center> LISTAR EL HISTORIAL DE TODAS LAS ACCIONES REALIZADAS EN UN REPOSITORIO GIT <br> LISTADO DESDE EL MAS NUEVO HASTA EL PRIMER CAMBIO</br> <br> PERMITE RECUPERAR COMMITS Y RAMAS BORRADAS ACCIDENTALMENTE USANDO EL HASH DEL COMMIT</br></center>

### OBTENER EL HISTORIAL DE LOS CAMBIOS EN EL REPOSITORIO

In [103]:
comando = 'git reflog'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git reflog
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
3b4edb5 HEAD@{0}: checkout: moving from hotfix_branch to main
168904c HEAD@{1}: cherry-pick: ADICIONADO ARCHIVO main.py 2023-03-25 12:43:02.603526
c9dcf21 HEAD@{2}: commit: ADICIONADO ARCHIVO info.txt 2023-03-25 12:43:09.024810
3b4edb5 HEAD@{3}: checkout: moving from main to hotfix_branch
3b4edb5 HEAD@{4}: checkout: moving from cherry_pick_branch to main
84ef7ab HEAD@{5}: commit: ADICIONADO ARCHIVO link.vs 2023-03-25 12:43:02.603526
b4326ed HEAD@{6}: commit: ADICIONADO ARCHIVO dataset.avro 2023-03-25 12:43:02.603526
f0d2cfd HEAD@{7}: commit: ADICIONADO ARCHIVO query.sql 2023-03-25 12:43:02.603526
197a215 HEAD@{8}: commit: ADICIONADO ARCHIVO utils.gif 2023-03-25 12:43:02.603526
5a261f5 HEAD@{9}: commit: ADICIONADO ARCHIVO addon.php 2023-03-25 12:43:02.603526

## RECUPERAR RAMA ELIMINADA

#### INSPECCIONAMOS EL CONTENIDO DE LA RAMA A RECUPERAR CON LA AYUDA DEL HASH DEL COMMIT

In [108]:
# adicionado archivo INFO.txt PARA USAR EN REFLOG
commit_a_recuperar = '39954ce'
comando = 'git show {commit_a_recuperar}'.format(commit_a_recuperar=commit_a_recuperar)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git show 39954ce
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
commit 39954cee9be9350269c28d17fa5e48ea877e4e2c
Author: PARADIGMA\QiDimMak <jorgecardona@utp.edu.co>
Date:   Sat Mar 25 12:40:04 2023 -0500

    adicionado archivo INFO.txt PARA USAR EN REFLOG

diff --git a/Dockerfile b/Dockerfile
deleted file mode 100644
index 5745a4b..0000000
--- a/Dockerfile
+++ /dev/null
@@ -1,2 +0,0 @@
-1679765949.490369 
-1679765951.585798 
diff --git a/Manifest.yaml b/Manifest.yaml
deleted file mode 100644
index 481ad32..0000000
--- a/Manifest.yaml
+++ /dev/null
@@ -1,2 +0,0 @@
-1679765949.628226 
-1679765951.701931 
diff --git a/addon.php b/addon.php
deleted file mode 100644
index ea1a081..0000000
--- a/addon.php
+++ /dev/null
@@ -1,2 +0,0 @@
-1679765950.327177 
-1679765952.42241 
diff --git a/calculos.c b/calculos.c
deleted f

#### OBTENEMOS EL CONTENIDO DEL COMMIT

In [109]:
comando = 'git checkout {commit_a_recuperar}'.format(commit_a_recuperar=commit_a_recuperar)
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout 39954ce
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
39954ce adicionado archivo INFO.txt PARA USAR EN REFLOG
5b478a2 PRUEBA numero 603120 de adicion de archivos y creacion de COMMIT 1679765953.390544
f4f41aa adicionados archivos de prueba
bd58137 adicionado archivo INFO.txt
##############################################################################################################



#### CREAMOS UNA NUEVA RAMA A PARTIR EL CONTENIDO DEL COMMIT RECUPERADO

In [110]:
rama = 'utils_recuperada'
comando = 'git branch {rama}'.format(rama=rama)
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git branch utils_recuperada
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git log --oneline
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
39954ce adicionado archivo INFO.txt PARA USAR EN REFLOG
5b478a2 PRUEBA numero 603120 de adicion de archivos y creacion de COMMIT 1679765953.390544
f4f41aa adicionados archivos de prueba
bd58137 adicionado archivo INFO.txt
##############################################################################################################



## ELIMINAR RAMA

In [111]:
# no se puede eliminar una rama si se esta trabajando en ese monento en ella
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git branch -D {rama}'.format(rama=rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

############################################# COMANDO A EJECUTAR #############################################
git checkout main
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Your branch is up to date with 'origin/main'.
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -D utils_recuperada
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
Deleted branch utils_recuperada (was 39954ce).
##############################################################################################################

############################################# COMANDO A EJECUTAR #############################################
git branch -a
####################################### RESULTADO DE COMANDO EJEC

### OBTENER EL HISTORIAL DE LOS CAMBIOS EN EL REPOSITORIO Y MOSTRAR LOS CAMBIOS DE CADA ARCHIVO EN ESOS CAMBIOS

In [112]:
comando = 'git reflog -g --patch'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

############################################# COMANDO A EJECUTAR #############################################
git reflog -g --patch
####################################### RESULTADO DE COMANDO EJECUTADO #######################################
3b4edb5 HEAD@{0}: checkout: moving from 39954cee9be9350269c28d17fa5e48ea877e4e2c to main
diff --git a/config.json b/config.json
new file mode 100644
index 0000000..e69de29
diff --git a/requirements.txt b/requirements.txt
new file mode 100644
index 0000000..e69de29
39954ce HEAD@{1}: checkout: moving from main to 39954ce
diff --git a/Dockerfile b/Dockerfile
deleted file mode 100644
index 5745a4b..0000000
--- a/Dockerfile
+++ /dev/null
@@ -1,2 +0,0 @@
-1679765949.490369 
-1679765951.585798 
diff --git a/Manifest.yaml b/Manifest.yaml
deleted file mode 100644
index 481ad32..0000000
--- a/Manifest.yaml
+++ /dev/null
@@ -1,2 +0,0 @@
-1679765949.628226 
-1679765951.701931 
diff --git a/addon.php b/addon.php
deleted file mode 100644
index ea1a081..0000000

# <center> TAGs <br>https://github.com/JorgeCardona/comandos_git</br></center>

In [ ]:
nombre_rama = 'tag_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

### CREAR UN TAG

In [ ]:
nombre_tag = 'v0.0.0'
comando = 'git tag  {nombre_tag} -m "versión Inicial"'.format(nombre_tag=nombre_tag)
ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### LISTAR LOS TAGs LOCALES

In [ ]:
comando = 'git tag'
ejecutar_comando_git(comando)

### ELIMINAR UN TAG LOCAL

In [ ]:
comando = 'git tag -d {nombre_tag}'.format(nombre_tag=nombre_tag)
ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### CREAR MULTIPLES TAGs

In [ ]:
lista_ramas_locales = obtener_lista_nombres_ramas_git()

for indice, rama in enumerate(lista_ramas_locales):
 
    #cambia de rama para tener el contenido especifico de esa rama en el TAG
    comando = 'git checkout {rama}"'.format(rama=rama)
    ejecutar_comando_git(comando)
    
    # genera el nombre que tendra el TAG
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)
    comando = 'git tag {nombre_tag} -m "version {indice} del TAG"'.format(nombre_tag=nombre_tag, indice=indice)
    ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### LISTAR LOS TAGs Y VER SU DESCRIPCION

In [ ]:
comando = 'git tag -n'
ejecutar_comando_git(comando)

### PUBLICAR UN TAG ESPECIFICO A UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
for indice, rama in enumerate([lista_ramas_locales[0]]):
    
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)
    comando = comando = 'git push origin {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)

### LISTAR LOS TAGs REMOTOS

In [ ]:
comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

### PUBLICAR TODOS LOS TAGs NO PUBLICADOS EN UN SOLO PUSH AL REPOSITORIO REMOTO

In [ ]:
comando = 'git push --tags'
ejecutar_comando_git(comando)

comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

### ELIMINAR LOS TAGs LOCALES Y REMOTOS

In [ ]:
for indice, rama in enumerate(lista_ramas_locales):
    
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)

    comando = 'git tag -d {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)

    comando = 'git push --delete origin {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)
    
comando = 'git tag'
ejecutar_comando_git(comando)

comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA DEL REPOSITORIO LOCAL Y REMOTO


In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)